## Wordvectors lab
LLM's and ChatGPT | Fall 2023 | McSweeney | CUNY Graduate Center

**Due:** October 8

* **DOWNLOAD** [Embeddings file for Part 2 here](https://drive.google.com/file/d/1NQ_7ERV3eiUnPCoIW7I8yi4TZMiu2cEQ/view?usp=sharing)
* Unzip the file
* Move it to the folder where you are writing your code. 


### Background
The purpose of this lab is to explore the process of transforming words into numerical representations, and then understanding those values spatially. By the end of this lab (and corresponding lecture), you should be comfortable with the fact that computers transform text into numbers based on the context in which they appear. 

There are **two** parts to this lab:

* **Part 1**: We'll train our own vectors on a very small corpus to understand vector similarity in a contextualized setting. We'll explore what vectors look like, how words are related, and the vocabulary of a model. 
* **Part 2**: GloVe word embeddings. GloVe is an unsupervised learning algorithm to calculate the vectors of the words in a corpus. We will use the pre-trained version that comes from Wikipedia articles. You can find [more about the GloVe vectors here](https://nlp.stanford.edu/projects/glove/). The goal of this section is to understand general word embeddings as trained on a very large corpus. 




#### Notes
You are not expected to understand the math or the code. We are using dimensionality reduction, which has a lot of uses including visualization and optimizing performance efficiency. For us, we are using dimensionality reduction for visualization. Word vectors create very large matrices. We can only visualize things on 2 or 3 dimension. So we reduce the dimensions. 

#### The book in the file
[From wikipedia:](https://en.wikipedia.org/wiki/Catching_Fire) Catching Fire is a 2009 dystopian young adult fiction novel by the American novelist Suzanne Collins, the second book in The Hunger Games series. As the sequel to the 2008 bestseller The Hunger Games, it continues the story of Katniss Everdeen and the post-apocalyptic nation of Panem. Following the events of the previous novel, a rebellion against the oppressive Capitol has begun, and Katniss and fellow tribute Peeta Mellark are forced to return to the arena in a special edition of the Hunger Games.

* `re` is the regular expression library
* `gensim` is for representing documents as vectors. It's the library built for making word vectors.

## Part 1: Gensim

In [ ]:
import re
import nltk
import numpy as np
import matplotlib.pyplot as plt
import random

from gensim.models import Word2Vec

from sklearn.decomposition import IncrementalPCA   
from sklearn.manifold import TSNE   


Load a file and replace the markup characters

In [ ]:
# read a file you have stored locally
# I added the Hunger Games for simplicity
file = open("hunger_games.txt", 'r').read()

# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
    file = file.replace(char, " ")

# check
print(file[:100])

Clean the text a little bit (tokenize, lowercase)

In [ ]:
# this is simplified for demonstration
def sample_clean_text(text: str):
    # step 1: tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)

    # step 2: tokenize each sentence into words
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

    # step 3: convert each word to lowercase
    tokenized_text = [[word.lower() for word in sent] for sent in tokenized_sentences]
    
    # return your tokens
    return tokenized_text

# call the function
tokens = sample_clean_text(text = file)

# check
print(tokens[:10])

Establish the [word2vec ](https://www.tensorflow.org/text/tutorials/word2vec#:~:text=word2vec%20is%20not%20a%20singular,downstream%20natural%20language%20processing%20tasks.)model. We've given it our list of tokens. We said that for each token, we want to define it in the context of 100 words. 

This means that each vector will have 100 numbers in it representing the relationship between this word and 100 others. 

The larger this number is, the better the model. The smaller the number, the worse the model. 

In [ ]:
model = Word2Vec(tokens,vector_size=100)

Now explore the entire vocabulary (note that only 1000 are represented because of the interface -- there are far more words in the vocab). Use this for ideas in the next section

In [ ]:
model.wv.key_to_index

Now we pick a word from the corpus (remember this is lowercase) and see what the vector associated with it is. This is not human-readable, but it's this list (array) of numbers that helps us define the word in context.

In [ ]:
model.wv.get_vector("capitol", norm=True)

Find all the words most similar to a word you've chosen. Notice that this similarity is not the same as the vector representation above. 

After you run this cell, make a note of the top 5 words. Go back and change the number of words in the vector and re-run the cells in between. Did it change?

In [ ]:
model.wv.most_similar('capitol')

Now see how similar some words are. Change these words if you change the text. 

In [ ]:
model.wv.similarity('katniss', 'girl')

In [ ]:
model.wv.similarity('peeta', 'home')

Finally visualize the result. There's a lot of code here, it's really just for visualization. The first part is the dimensionality reduction (using t-SNE). The second part puts it on a visualization. 


Spend more time exploring the visualization than breaking down the code. 

The visualization that results is very hard to see. If you are in a Jupyter Notebook, there are tools in the upper right corner of the cell that has the visualization in it (they appear on hover). You can zoom in on the visualization to see the words and the relationships between the words. 

In [ ]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

In [ ]:
def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

## Part 2 : GloVe

**BEFORE you move on, did you:**

* DOWNLOAD the [Embeddings file for Part 2 here](https://drive.google.com/file/d/1NQ_7ERV3eiUnPCoIW7I8yi4TZMiu2cEQ/view?usp=sharing)
* Unzip the file
* Move it to the folder where you are writing your code. 

In [ ]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

There are a few files in the `glove/6B` folder. These are the embeddings. You can open them and see them in any text editor. 50d is the smallest one. Each word has 50 dimensions. This means it is defined in terms of 50 other words. 

This lab uses the 100d file. If it's running slower than you like, feel free to use 50d instead. If you want more accurate word relationships, use one of the larger files. 

In [ ]:
#establish an empty dictionary
embeddings_dict = {}

#open the file and read it into the dictionary
with open("glove.6B/glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [ ]:
#find the Euclidean distance between the vectors for words and 1 or more other words.
#sort the resulting word distances.
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: 
                  spatial.distance.euclidean(embeddings_dict[word], embedding))

Now try it out - pick some words to experiment with. This experimentation section is the most important part of Part 2.

In [ ]:
print(find_closest_embeddings(
    embeddings_dict["dog"]
)[:20])

In [ ]:
print(find_closest_embeddings(
    embeddings_dict["cat"]
)[:20])

Now pick 2 words and see how it changes. Note that the `+` actually indicates that the vectors are being added. This may have slightly unexpected results.

In [ ]:
print(find_closest_embeddings(
    embeddings_dict["dog"] + embeddings_dict["cat"]
)[:20])

See what happens with a 3rd. These embeddings are basically being triangulated - and this list is what is nearest to the center of these 3 points.

In [ ]:
print(find_closest_embeddings(
    embeddings_dict["dog"] + embeddings_dict["cat"] + embeddings_dict["pet"]
)[:20])

Now transform the dictionary into an array so that we can do dimensionality reduction so we can visualize the words

In [ ]:
words =  list(embeddings_dict.keys())
vectors = [embeddings_dict[word] for word in words]
X = np.asarray(vectors)

Now do the dimensionality reduction with t-SNE.

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
Y = tsne.fit_transform(X[:1000])

Plot the points

In [ ]:
plt.scatter(Y[:, 0], Y[:, 1])

Annotate a few of the words. 

In [ ]:
for label, x, y in zip(words, X[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")
plt.show()